In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

### API-Setup

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 46.9481,
	"longitude": 7.4474,
	"current": ["temperature_2m", "is_day", "precipitation", "rain", "showers", "snowfall", "cloud_cover", "surface_pressure"],
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "weather_code", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility", "uv_index", "uv_index_clear_sky", "is_day", "sunshine_duration"],
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "sunrise", "sunset", "sunshine_duration", "precipitation_sum", "rain_sum", "showers_sum", "snowfall_sum", "precipitation_hours", "precipitation_probability_max"],
	"timezone": "Europe/Berlin",
	"forecast_days": 3,
	"models": "best_match"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 46.939998626708984°N 7.440000057220459°E
Elevation 554.0 m asl
Timezone b'Europe/Berlin' b'CET'
Timezone difference to GMT+0 3600 s


### Current Values

In [3]:
# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_is_day = current.Variables(1).Value()
current_precipitation = current.Variables(2).Value()
current_rain = current.Variables(3).Value()
current_showers = current.Variables(4).Value()
current_snowfall = current.Variables(5).Value()
current_cloud_cover = current.Variables(6).Value()
current_surface_pressure = current.Variables(7).Value()

print(f"Current time {current.Time()}")
print(f"Current temperature_2m {current_temperature_2m}")
print(f"Current is_day {current_is_day}")
print(f"Current precipitation {current_precipitation}")
print(f"Current rain {current_rain}")
print(f"Current showers {current_showers}")
print(f"Current snowfall {current_snowfall}")
print(f"Current cloud_cover {current_cloud_cover}")
print(f"Current surface_pressure {current_surface_pressure}")

Current time 1710842400
Current temperature_2m 9.25
Current is_day 1.0
Current precipitation 0.0
Current rain 0.0
Current showers 0.0
Current snowfall 0.0
Current cloud_cover 100.0
Current surface_pressure 954.2854614257812


### Hourly Data

In [6]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_showers = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(7).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(8).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(9).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(12).ValuesAsNumpy()
hourly_visibility = hourly.Variables(13).ValuesAsNumpy()
hourly_uv_index = hourly.Variables(14).ValuesAsNumpy()
hourly_uv_index_clear_sky = hourly.Variables(15).ValuesAsNumpy()
hourly_is_day = hourly.Variables(16).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(17).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["showers"] = hourly_showers
hourly_data["snowfall"] = hourly_snowfall
hourly_data["weather_code"] = hourly_weather_code
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["visibility"] = hourly_visibility
hourly_data["uv_index"] = hourly_uv_index
hourly_data["uv_index_clear_sky"] = hourly_uv_index_clear_sky
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,precipitation_probability,precipitation,rain,showers,snowfall,weather_code,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,uv_index,uv_index_clear_sky,is_day,sunshine_duration
0,2024-03-18 23:00:00+00:00,7.998,99.0,15.0,0.0,0.0,0.0,0.0,45.0,953.723999,95.0,90.0,29.0,0.0,24140.0,0.0,0.0,0.0,0.0
1,2024-03-19 00:00:00+00:00,7.748,98.0,0.0,0.0,0.0,0.0,0.0,45.0,953.854370,96.0,92.0,47.0,0.0,24140.0,0.0,0.0,0.0,0.0
2,2024-03-19 01:00:00+00:00,7.098,99.0,0.0,0.0,0.0,0.0,0.0,45.0,953.333252,89.0,74.0,54.0,8.0,24140.0,0.0,0.0,0.0,0.0
3,2024-03-19 02:00:00+00:00,6.248,100.0,0.0,0.0,0.0,0.0,0.0,45.0,953.140015,92.0,91.0,0.0,50.0,24140.0,0.0,0.0,0.0,0.0
4,2024-03-19 03:00:00+00:00,5.898,100.0,0.0,0.0,0.0,0.0,0.0,45.0,953.714294,100.0,100.0,0.0,48.0,24140.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2024-03-21 18:00:00+00:00,11.554,94.0,61.0,0.0,0.0,0.0,0.0,2.0,960.132202,76.0,76.0,0.0,0.0,24140.0,0.1,0.1,0.0,0.0
68,2024-03-21 19:00:00+00:00,10.554,94.0,49.0,0.0,0.0,0.0,0.0,2.0,960.097351,50.0,50.0,0.0,0.0,24140.0,0.0,0.0,0.0,0.0
69,2024-03-21 20:00:00+00:00,9.554,96.0,38.0,0.0,0.0,0.0,0.0,45.0,960.154480,44.0,44.0,0.0,0.0,24140.0,0.0,0.0,0.0,0.0
70,2024-03-21 21:00:00+00:00,8.354,99.0,26.0,0.0,0.0,0.0,0.0,45.0,960.257935,44.0,40.0,0.0,4.0,24140.0,0.0,0.0,0.0,0.0


### Daily Data

In [7]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_sunrise = daily.Variables(3).ValuesAsNumpy()
daily_sunset = daily.Variables(4).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(5).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(6).ValuesAsNumpy()
daily_rain_sum = daily.Variables(7).ValuesAsNumpy()
daily_showers_sum = daily.Variables(8).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(9).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(11).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["showers_sum"] = daily_showers_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe

,date,weather_code,temperature_2m_max,temperature_2m_min,sunrise,sunset,sunshine_duration,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,precipitation_probability_max
0,2024-03-18 23:00:00+00:00,45.0,14.798,5.598,0,0,26118.875000,0.0,0.0,0.0,0.0,0.0,15.0
1,2024-03-19 23:00:00+00:00,45.0,17.598,2.948,0,0,36029.546875,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-03-20 23:00:00+00:00,81.0,14.454,7.448,0,0,17722.107422,11.4,6.9,4.6,0.0,15.0,87.0
